In [14]:
import matplotlib.pyplot as plt
import seaborn
import pandas
import sys
import numpy as np
from scipy import stats
from collections import Counter

# this tells seaborn and matplotlib to generate plots inline in the notebook
%matplotlib inline  

# these two lines allow you to control the figure size
%pylab inline
pylab.rcParams['figure.figsize'] = (16.0, 8.0) 
 
ur = pandas.read_csv('data/movie/user_ratedmovies_train.dat','\t')
predictions = pandas.read_csv('data/movie/predictions.dat', '\t')

predictions = predictions[7119:len(predictions)]

print len(predictions)

# create a test/train split
 
all_inds = np.random.permutation( range(0,len(ur)) )
## VALIDATION SET FOR TESTING
test_inds = all_inds[0:10]
train_inds = all_inds[600000:len(ur)]

#train_inds = all_inds
 
ur_test = ur.iloc[ test_inds ]
ur_train = ur.iloc[ train_inds ]

print len(ur_train)

Populating the interactive namespace from numpy and matplotlib
77881
170598


In [15]:
def get_two_user_data(userID1, userID2):
    return ur_train.query("userID == %d or userID == %d" % (userID1, userID2))

def get_intersection_list(user_data):
    movies_rated = user_data['movieID']
    cnt = Counter(movies_rated)
    return [k for k, v in cnt.iteritems() if v > 1]    

def get_intersection_data(user_data, intersection_list):
    return user_data[user_data['movieID'].isin(intersection_list)]

def most_common(lst):
    return max(set(lst), key=lst.count)

def get_relationship_strength(user1, user2):
    user_data = get_two_user_data(user1, user2)
    intersection_list = get_intersection_list(user_data)
    if len(intersection_list) == 0: return 1
    intersection_data = get_intersection_data(user_data, intersection_list)
    
    user1Ratings = intersection_data[intersection_data['userID'] == user1].sort_values('movieID')['rating']
    user2Ratings = intersection_data[intersection_data['userID'] == user2].sort_values('movieID')['rating']
    
    sse = get_euclid(user1Ratings, user2Ratings)
    return sse / len(intersection_list)

def guess_with_k_neighbors(relationshipList, movieData, k=5):
    closest = np.array(relationshipList).argsort()[:k]
    ratings = []
    for index in closest:
        ratings.append( movieData.iloc[index]['rating'] )
    if ratings:
        return most_common(ratings)
    else:
        return 4.0 #make a guess
    
def get_SSE(guess, actual):
    return (guess - actual)  ** 2

def get_RMSE(guessList, actualList):
    from sklearn.metrics import mean_squared_error
    return mean_squared_error(actualList, guessList)**0.5

def get_euclid(user1Ratings, user2Ratings):
    return np.sum(np.sqrt((np.array(user1Ratings) - np.array(user2Ratings)) ** 2))

In [16]:
#print "Test Data Len: ", len(predictions)
sse = 0
iteration = 0 

guesses = []


for some_index, userData in predictions.iterrows():
    movieData =  ur_train[ur_train['movieID'] == userData.movieID]
    userIDList = movieData['userID']
    #print "Train Data Len: ", len(userIDList)

    relationshipList = []
    index = 0
    sys.stdout.write("\r Iter: %d" % (iteration))
    for comparisonID in userIDList:
        #sys.stdout.write("\r Iter: %d, Index: %d, ID: %d, Compared ID: %d" % (iteration, index, userData.userID, comparisonID))
        if userData.userID != comparisonID:
            relationshipList.append( get_relationship_strength(userData.userID, comparisonID) )
        index += 1
    iteration += 1
    guess = guess_with_k_neighbors(relationshipList, movieData, k=5)
    guesses.append( guess )
    #print "\n\nGuess: ", guess
    #print "\nActual: ", userData.rating, "\n\n"
    #sse += get_SSE(guess, userData.rating)


 Iter: 77880

In [17]:
sfile = open( 'movie_predictions_extension.csv', 'wb' )
sfile.write( '"testID","predicted rating"\n' )
for index in range( 0 , len(predictions) ):
    sfile.write( '%d,%.2f\n' % ( predictions.iloc[index].testID , guesses[index] ) )  # add one for one-based indexing
sfile.close()